<a href="https://colab.research.google.com/github/Ghaissen/Zoosanitary-deteciton-LLM-/blob/main/web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk import ngrams
import nltk
import re
import string
import urllib.request
import requests
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from datetime import datetime
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate
from sklearn.decomposition import PCA,LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from textblob import TextBlob,Word
from urllib.request import Request
from bs4 import BeautifulSoup

In [ ]:
# Charger le dataset
data_path = "/content/drive/MyDrive/pcd/data_processed_PCD.csv"
df = pd.read_csv(data_path)
print(df.head())

                                             Article language  probability  \
0  Crimean-Congo Haemorrhagic Fever Cases Increas...       en     0.999998   
1  State confirms potentially deadly EEE virus in...       en     0.999995   
2  Nagaland officially declared as Lumpy Skin Dis...       en     0.999997   
3  Nagaland officially declared as Lumpy Skin Dis...       en     0.999998   
4  Lumpy skin disease (LSD) is a notifiable disea...       en     1.000000   

                                      processed_text  
0  Crimean Congo Haemorrhagic Fever case Increase...  
1  state confirm potentially deadly EEE virus Bar...  
2  Nagaland officially declare Lumpy Skin Disease...  
3  Nagaland officially declare Lumpy Skin Disease...  
4  lumpy skin disease LSD notifiable disease acco...  


In [ ]:
!pip install requests beautifulsoup4 lxml

In [ ]:
def getlinks(soup, base_url):
    l = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        # Check if the href is a full URL or a relative path
        if not href.startswith('http'):
            href = base_url + href  # Concatenate with base URL if it's a relative path
        l.append(href)
    return l

def getdate(soup):
    div = soup.find('label', class_='label')
    return div.text if div else "No Date Found"
#
#def getname(soup):
 #   l = []
 #   div = soup.find('a', class_ = 'bbp-author-name')
 #   return([div.text])
#
def gettext(soup):
    p = soup.find('section', class_='rte w-max')
    return p.text if p else "No Text Found"
#

def get_yearly_links(base_url):
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    yearly_links = [a['href'] for a in soup.find_all('a', class_='nav-link')]
    return yearly_links


sitelist = []
site = 'https://www.ssi.dk/aktuelt/nyheder'
hdr = {'User-Agent': 'Mozilla/5.0'}
yearly_links = get_yearly_links(site)

df = pd.DataFrame()

# This assumes each yearly link contains pages that need to be scraped
for year_link in yearly_links:
    print(f"Scraping {year_link}")
    year_response = requests.get(year_link, headers=hdr)
    year_soup = BeautifulSoup(year_response.text, 'html.parser')
    links = getlinks(year_soup, site)
    for link in links:
      link_response = requests.get(link, headers=hdr)
      if link_response.status_code == 200:
        link_soup = BeautifulSoup(link_response.text, 'html.parser')
        date = getdate(link_soup)
        text = gettext(link_soup)
        d = [{'DATE': date, 'TEXT': text, 'STATE': "no event"}]
        df = df.append(pd.DataFrame(d), ignore_index=True)
df.to_csv('/content/drive/MyDrive/pcd/articles.csv')

In [ ]:
!pip install googletrans==4.0.0-rc1


In [ ]:
from googletrans import Translator


In [ ]:
# Charger le dataset
data_path2 = "/content/drive/MyDrive/pcd/articles.csv"
df2 = pd.read_csv(data_path2)
print(df2.shape)

(5761, 4)


In [ ]:
df2 = df2[df2['DATE'] != 'No Date Found']

df2 = df2[df2['TEXT'] != 'No Text Found']

df2.to_csv('/content/drive/MyDrive/pcd/articles_v2.csv', index=False)
print(df2.shape)

(3805, 4)


In [ ]:

df2 = df2.sample(n=25, random_state=1)  # Changez 'random_state' pour différents échantillons
df2.to_csv('/content/drive/MyDrive/pcd/articles_25.csv', index=False)
print(df2.shape)

(25, 4)


In [ ]:
data_path3 = "/content/drive/MyDrive/pcd/articles_25.csv"
df3 = pd.read_csv(data_path3)

In [ ]:

# Initialize translator
translator = Translator()

def safe_translate(text, dest_language):
    try:
        translation = translator.translate(text, dest=dest_language)
        if translation and hasattr(translation, 'text'):
            return translation.text
    except Exception as e:
        print(f"An error occurred: {e}")
    return text  # Return the original text if translation fails

# Assuming 'df' is your DataFrame and it has a column named 'TEXT' with Danish texts
df3['TEXT_EN'] = df3['TEXT'].apply(lambda text: safe_translate(text, 'en'))
df3['TEXT_FR'] = df3['TEXT'].apply(lambda text: safe_translate(text, 'fr'))

df3.to_csv('/content/drive/MyDrive/pcd/translated_articles.csv')

An error occurred: the JSON object must be str, bytes or bytearray, not NoneType
An error occurred: the JSON object must be str, bytes or bytearray, not NoneType
An error occurred: the JSON object must be str, bytes or bytearray, not NoneType
An error occurred: the JSON object must be str, bytes or bytearray, not NoneType
An error occurred: the JSON object must be str, bytes or bytearray, not NoneType
An error occurred: list index out of range
An error occurred: the JSON object must be str, bytes or bytearray, not NoneType


In [ ]:
def get_disease_info(soup):
    diseases_data = []
    h3_tags = soup.find_all('h3')

    for h3_tag in h3_tags:
        disease_name = h3_tag.get_text(strip=True)
        content_texts = []
        text_content=''

        # Iterate over the next siblings of the current h3 tag
        for sibling in h3_tag.next_siblings:
            if sibling.name == 'h3':
                break  # Stop if the next h3 tag is found
            if sibling.name == 'p':
              text_content=text_content+' '
              content_texts.append(sibling.get_text(strip=True))
            elif sibling.name == 'ul':
              text_content=text_content+' '
              list_items = sibling.find_all('li')
              list_contents = ' '.join(li.get_text(strip=True) for li in list_items)
              content_texts.append(list_contents)

        # Concatenate disease name with its content texts
        full_text = ' '.join([disease_name] + content_texts)
        diseases_data.append(full_text)

    return diseases_data

site = 'https://ar.wikipedia.org/wiki/%D9%85%D8%B1%D8%B6_%D8%A7%D9%84%D8%AF%D9%88%D8%A7%D8%AC%D9%86#%D8%A8%D8%B9%D8%B6_%D8%A7%D9%85%D8%B1%D8%A7%D8%B6_%D8%A7%D9%84%D8%AF%D9%88%D8%A7%D8%AC%D9%86'
hdr = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(site, headers=hdr)
soup = BeautifulSoup(response.text, 'html.parser')

# Get disease info from the page
disease_info_list = get_disease_info(soup)

# Create DataFrame from the scraped data
df4 = pd.DataFrame(disease_info_list, columns=['TEXT'])

# Add a state column with a default value
df4['STATE'] = "no event"

# Save the DataFrame to a CSV file
csv_path = '/content/drive/MyDrive/pcd/articles_animal_disease.csv'  # Modify as needed for your path
df4.to_csv(csv_path, index=False)


In [ ]:
def getlinks(soup, base_url):
    l = []
    table = soup.find('table', {'width': '60%'})
    for link in table.find_all('a', href=True):
        href = link['href']
        # Prepend base_url if href is a relative path
        if not href.startswith('http'):
            href = base_url + href
        l.append(href)
    return l

def gettext(soup):
    content_div = soup.find('div', class_='mw-content-rtl mw-parser-output')
    text_content = []
    if content_div:
        for elem in content_div.children:
            if elem.name == 'meta' and elem.get('property', '') == 'mw:PageProp/toc':
                break
            if elem.name == 'p':
              text_content.append(elem.get_text(strip=True))
    return ' '.join(text_content) if text_content else "No Text Found"

# Base URL definition for Wikipedia
base_url = 'https://ar.wikipedia.org'
site = base_url + '/wiki/%D8%B7%D8%A8_%D8%A8%D9%8A%D8%B7%D8%B1%D9%8A'
hdr = {'User-Agent': 'Mozilla/5.0'}

# Request the main page to get links from
response = requests.get(site, headers=hdr)
soup = BeautifulSoup(response.text, 'html.parser')
links = getlinks(soup, base_url)

# Data list to hold the dictionaries before creating DataFrame
data_list = []

# Loop through each link and scrape the text
for link in links:
    print(f"Scraping {link}")
    link_response = requests.get(link, headers=hdr)
    if link_response.status_code == 200:
        link_soup = BeautifulSoup(link_response.text, 'html.parser')
        text = gettext(link_soup)
        data_list.append({'TEXT': text, 'STATE': "no event"})

# Create DataFrame from the list of data
df5 = pd.DataFrame(data_list)

# Save the DataFrame to a CSV file
# Change the path according to your system's directory structure
csv_path = '/content/drive/MyDrive/pcd/animal_diseases_2.csv'
df5.to_csv(csv_path, index=False)


Scraping https://ar.wikipedia.org/wiki/%D8%AD%D9%85%D9%89_%D8%A7%D9%84%D9%88%D8%A7%D8%AF%D9%8A_%D8%A7%D9%84%D9%85%D8%AA%D8%B5%D8%AF%D8%B9
Scraping https://ar.wikipedia.org/wiki/%D8%A5%D9%86%D9%81%D9%84%D9%88%D9%86%D8%B2%D8%A7_%D8%A7%D9%84%D8%B7%D9%8A%D9%88%D8%B1
Scraping https://ar.wikipedia.org/wiki/%D8%AD%D9%85%D9%89_%D9%82%D9%84%D8%A7%D8%B9%D9%8A%D8%A9
Scraping https://ar.wikipedia.org/wiki/%D8%AF%D8%A7%D8%A1_%D8%A7%D9%84%D9%83%D9%84%D8%A8
Scraping https://ar.wikipedia.org/wiki/%D8%AC%D8%B1%D8%A8_%D8%A7%D9%84%D8%A5%D8%A8%D9%84
Scraping https://ar.wikipedia.org/wiki/%D9%86%D8%BA%D9%81
Scraping https://ar.wikipedia.org/wiki/%D8%AF%D8%A7%D8%A1_%D8%A7%D9%84%D9%85%D8%AA%D9%88%D8%B1%D9%82%D8%A7%D8%AA
Scraping https://ar.wikipedia.org/wiki/%D8%AF%D8%A7%D8%A1_%D8%A7%D9%84%D8%BA%D9%88%D9%85%D8%A8%D9%88%D8%B1%D9%88
Scraping https://ar.wikipedia.org/wiki/%D8%A5%D9%86%D9%81%D9%84%D9%88%D9%86%D8%B2%D8%A7_%D8%A7%D9%84%D8%AE%D9%8A%D9%88%D9%84
Scraping https://ar.wikipedia.org/wiki/%D9%85%D8%B1%D8%